In [47]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler, RobustScaler
from collections import OrderedDict

In [2]:
import os, sys
sys.path.append(os.getcwd())

In [3]:
from _element import feature_control as ft_c
from _element import varr
from _element import calculations as calc
from _element.result_control import Result

from _alg.prophet import Prophet_timeseries

from _evaluation.cross_validation import Cross_Validation
from _evaluation import compare

In [4]:
INPUT_FILENAME= 'KPP일별투입(10_17)_raw.xlsx'

In [5]:
df_raw= ft_c.xlsx_opener(varr.PJ_DIR+varr.DF_DIR, INPUT_FILENAME)
df_temp= ft_c.xlsx_opener(varr.PJ_DIR+varr.TEMP_DATA_DIR, 'temp_data_merged.xlsx')

In [6]:
y_col= df_raw['raw'].columns.values.tolist()

In [7]:
try:
    y_col.remove('ds')
except ValueError:
    pass

In [8]:
x_col= df_temp['raw'].columns.values.tolist()

In [9]:
df_raw['raw']['ds']= pd.to_datetime(df_raw['raw']['ds'], box=True, format= '%Y/%m/%d', exact=True)
df_temp['raw']['ds']= pd.to_datetime(df_temp['raw']['ds'], box=True, format= '%Y/%m/%d', exact=True)
df_raw['raw']= pd.merge(df_raw['raw'], df_temp['raw'], how='inner', on='ds')

In [11]:
ft_c.add_division(df_raw, 'raw', 7, start_num= 3)

In [12]:
ft_c.divide_by(df_raw, 'raw', 'seasonality_7,3')

In [13]:
df_raw

{(0,):              ds       y   temp_max   temp_min  rain_amount  seasonality_7,3
 4    2010-07-05   65049  30.600000  21.799999          0.0                0
 11   2010-07-12   67680  28.299999  22.200001          0.0                0
 18   2010-07-19   73816  29.299999  24.200001          0.0                0
 25   2010-07-26   75731  27.600000  23.400000          6.0                0
 32   2010-08-02   67290  31.600000  24.000000          1.0                0
 39   2010-08-09   67691  32.799999  26.400000          0.0                0
 46   2010-08-16   70947  26.500000  21.000000          0.0                0
 53   2010-08-23   71422  28.900000  23.500000         13.0                0
 60   2010-08-30   68075  29.799999  22.299999          0.0                0
 67   2010-09-06   74425  29.700001  23.000000          0.0                0
 74   2010-09-13   75663  27.400000  21.100000          0.0                0
 81   2010-09-20   56183  25.400000  19.400000          4.0           

In [14]:
r= Result()

In [18]:
cv= Cross_Validation(df_raw)

In [19]:
cv.slice('y', x_col, forecastday= varr.FORECASTDAY//7)

In [20]:
cv._data

{'(0,)_0': {'test':              ds       y  temp_max  temp_min  rain_amount  seasonality_7,3
  2706 2017-11-27  117480       8.1       1.8          0.0                0,
  'testX':              ds  temp_max  temp_min  rain_amount
  2706 2017-11-27       8.1       1.8          0.0,
  'testY': 2706    117480
  Name: y, dtype: int64,
  'train':              ds       y   temp_max   temp_min  rain_amount  seasonality_7,3
  4    2010-07-05   65049  30.600000  21.799999          0.0                0
  11   2010-07-12   67680  28.299999  22.200001          0.0                0
  18   2010-07-19   73816  29.299999  24.200001          0.0                0
  25   2010-07-26   75731  27.600000  23.400000          6.0                0
  32   2010-08-02   67290  31.600000  24.000000          1.0                0
  39   2010-08-09   67691  32.799999  26.400000          0.0                0
  46   2010-08-16   70947  26.500000  21.000000          0.0                0
  53   2010-08-23   71422  28.900

In [21]:
prpt_withweekday= Prophet_timeseries(cv= True)

In [22]:
prpt_withweekday.add_model('weekday', 'week', regressor= None)

In [23]:
prpt_withweekday.fit('weekday', txs_traintest= cv._data)

In [24]:
prpt_withweekday.forecast('weekday', forecast_dict= r.forecast_dict)

C:\Studying\Project_Nextop\nextop-engine\nextop_engine\_alg\prophet.py:174: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  result_forecast['ds'], box=True, format= '%Y/%m/%d', exact=True


In [29]:
r.forecast_dict

OrderedDict([('weekday__0_merged',
              {'MAPE': 45.713163163531796,
               'MAPE_div_std': 73.606459015732284,
               'MAPE_with_std': 20.22663478899732,
               'RMSE': 5794.6743253523309,
               'forecast':           ds       y  temp_max  temp_min  rain_amount  seasonality_7,3  \
               0 2017-11-24  120647       2.9      -2.5          0.1                4   
               0 2017-11-25   67756       6.4       0.8         15.7                5   
               0 2017-11-26     100       6.4      -2.4          0.0                6   
               0 2017-11-27  117480       8.1       1.8          0.0                0   
               0 2017-11-28  122192       9.8       4.2          1.2                1   
               0 2017-11-29  120685       5.7      -3.2          0.0                2   
               0 2017-11-30  108386       1.8      -7.1          0.0                3   
               
                           yhat  
   

In [26]:
r.merge_result(m_name= ['_0', '_1', '_2', '_3', '_4'], m_title= 'weekday')

{'forecast':           ds       y  temp_max  temp_min  rain_amount  seasonality_7,3  \
0 2017-11-24  120647       2.9      -2.5          0.1                4   
0 2017-11-25   67756       6.4       0.8         15.7                5   
0 2017-11-26     100       6.4      -2.4          0.0                6   
0 2017-11-27  117480       8.1       1.8          0.0                0   
0 2017-11-28  122192       9.8       4.2          1.2                1   
0 2017-11-29  120685       5.7      -3.2          0.0                2   
0 2017-11-30  108386       1.8      -7.1          0.0                3   

            yhat  
0  124592.326772  
0   62672.072598  
0    -191.516617  
0  113686.716103  
0  121378.489147  
0  122630.675957  
0  121603.957221  }
{'forecast':           ds       y   temp_max   temp_min  rain_amount  seasonality_7,3  \
0 2017-09-15  146743  27.299999  17.200001          0.0                4   
0 2017-09-16   78827  26.100000  19.200001          0.0                5   


In [27]:
r.err_rate(m_name= 'weekday')
r.print_err_rate()

RMSE with segment weekday__0_merged: 5794.6743
MAPE with segment weekday__0_merged: 45.7132
MAPE_with_std with segment weekday__0_merged: 20.2266
MAPE_div_std with segment weekday__0_merged: 73.6065
sMAPE with segment weekday__0_merged: 4.3957


RMSE with segment weekday__1_merged: 10462.4891
MAPE with segment weekday__1_merged: 21.5721
MAPE_with_std with segment weekday__1_merged: 24.4708
MAPE_div_std with segment weekday__1_merged: 235.6025
sMAPE with segment weekday__1_merged: 9.2321


RMSE with segment weekday__2_merged: 6768.3937
MAPE with segment weekday__2_merged: 43.0813
MAPE_with_std with segment weekday__2_merged: 20.8105
MAPE_div_std with segment weekday__2_merged: 82.9035
sMAPE with segment weekday__2_merged: 4.8596


RMSE with segment weekday__3_merged: 25615.0623
MAPE with segment weekday__3_merged: 37.9641
MAPE_with_std with segment weekday__3_merged: 37.1187
MAPE_div_std with segment weekday__3_merged: 82.7611
sMAPE with segment weekday__3_merged: 22.9499


RMSE with se

In [30]:
r.avg_result(m_name= 'weekday')

{'forecast': [          ds       y  temp_max  temp_min  rain_amount  seasonality_7,3  \
0 2017-11-24  120647       2.9      -2.5          0.1                4   
0 2017-11-25   67756       6.4       0.8         15.7                5   
0 2017-11-26     100       6.4      -2.4          0.0                6   
0 2017-11-27  117480       8.1       1.8          0.0                0   
0 2017-11-28  122192       9.8       4.2          1.2                1   
0 2017-11-29  120685       5.7      -3.2          0.0                2   
0 2017-11-30  108386       1.8      -7.1          0.0                3   

            yhat  
0  124592.326772  
0   62672.072598  
0    -191.516617  
0  113686.716103  
0  121378.489147  
0  122630.675957  
0  121603.957221  ,           ds       y   temp_max   temp_min  rain_amount  seasonality_7,3  \
0 2017-09-15  146743  27.299999  17.200001          0.0                4   
0 2017-09-16   78827  26.100000  19.200001          0.0                5   
0 2017-09-17

In [31]:
r.print_err_rate()

RMSE with segment weekday_result: 13547.0090
MAPE with segment weekday_result: 36.0416
MAPE_with_std with segment weekday_result: 25.3174
MAPE_div_std with segment weekday_result: 140.7680
sMAPE with segment weekday_result: 11.8902




In [106]:
df_raw= ft_c.xlsx_opener(varr.PJ_DIR+varr.DF_DIR, INPUT_FILENAME)
df_temp= ft_c.xlsx_opener(varr.PJ_DIR+varr.TEMP_DATA_DIR, 'temp_data_merged.xlsx')

In [107]:
y_col= df_raw['raw'].columns.values.tolist()

In [108]:
try:
    y_col.remove('ds')
except ValueError:
    pass

In [109]:
x_col= df_temp['raw'].columns.values.tolist()

In [110]:
df_raw['raw']['ds']= pd.to_datetime(df_raw['raw']['ds'], box=True, format= '%Y/%m/%d', exact=True)
df_temp['raw']['ds']= pd.to_datetime(df_temp['raw']['ds'], box=True, format= '%Y/%m/%d', exact=True)
df_raw['raw']= pd.merge(df_raw['raw'], df_temp['raw'], how='inner', on='ds')

In [111]:
std_scaler= StandardScaler()

In [112]:
df_raw['raw']['y']= std_scaler.fit_transform(df_raw['raw']['y'].reshape(-1,1))

C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: reshape is deprecated and will raise in a subsequent release. Please use .values.reshape(...) instead
  """Entry point for launching an IPython kernel.
C:\Anaconda3\lib\site-packages\sklearn\utils\validation.py:475: DataConversionWarning: Data with input dtype int64 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)


In [113]:
df_raw['raw']

,ds,y,temp_max,temp_min,rain_amount
0,2010-07-01,0.088409,30.500000,24.900000,0.0
1,2010-07-02,0.085019,25.799999,22.700001,70.0
2,2010-07-03,-0.698822,25.600000,22.600000,1.0
3,2010-07-04,-1.898451,29.700001,23.100000,0.0
4,2010-07-05,-0.279418,30.600000,21.799999,0.0
5,2010-07-06,0.206161,31.000000,22.400000,0.0
6,2010-07-07,0.111251,29.600000,21.900000,0.0
7,2010-07-08,-0.054766,29.000000,21.299999,0.0
8,2010-07-09,0.102979,29.299999,21.799999,0.0
9,2010-07-10,-0.671680,29.100000,22.900000,0.0


In [114]:
ft_c.add_division(df_raw, 'raw', 7, start_num= 3)

In [115]:
ft_c.divide_by(df_raw, 'raw', 'seasonality_7,3')

In [116]:
df_raw

{(0,):              ds         y   temp_max   temp_min  rain_amount  seasonality_7,3
 4    2010-07-05 -0.279418  30.600000  21.799999          0.0                0
 11   2010-07-12 -0.212864  28.299999  22.200001          0.0                0
 18   2010-07-19 -0.057649  29.299999  24.200001          0.0                0
 25   2010-07-26 -0.009208  27.600000  23.400000          6.0                0
 32   2010-08-02 -0.222730  31.600000  24.000000          1.0                0
 39   2010-08-09 -0.212586  32.799999  26.400000          0.0                0
 46   2010-08-16 -0.130223  26.500000  21.000000          0.0                0
 53   2010-08-23 -0.118207  28.900000  23.500000         13.0                0
 60   2010-08-30 -0.202873  29.799999  22.299999          0.0                0
 67   2010-09-06 -0.042244  29.700001  23.000000          0.0                0
 74   2010-09-13 -0.010928  27.400000  21.100000          0.0                0
 81   2010-09-20 -0.503690  25.400000  19.4000

In [117]:
cv2= Cross_Validation(df_raw)

In [118]:
cv2.slice('y', x_col, forecastday= varr.FORECASTDAY//7)

In [119]:
cv2._data

{'(0,)_0': {'test':              ds         y  temp_max  temp_min  rain_amount  seasonality_7,3
  2706 2017-11-27  1.046867       8.1       1.8          0.0                0,
  'testX':              ds  temp_max  temp_min  rain_amount
  2706 2017-11-27       8.1       1.8          0.0,
  'testY': 2706    1.046867
  Name: y, dtype: float64,
  'train':              ds         y   temp_max   temp_min  rain_amount  seasonality_7,3
  4    2010-07-05 -0.279418  30.600000  21.799999          0.0                0
  11   2010-07-12 -0.212864  28.299999  22.200001          0.0                0
  18   2010-07-19 -0.057649  29.299999  24.200001          0.0                0
  25   2010-07-26 -0.009208  27.600000  23.400000          6.0                0
  32   2010-08-02 -0.222730  31.600000  24.000000          1.0                0
  39   2010-08-09 -0.212586  32.799999  26.400000          0.0                0
  46   2010-08-16 -0.130223  26.500000  21.000000          0.0                0
  53   20

In [120]:
prpt_withscaler= Prophet_timeseries(cv= True)

In [121]:
prpt_withscaler.add_model('StdSc', 'week', regressor= ['temp_max', 'temp_min', 'rain_amount'])

In [122]:
prpt_withscaler.fit('StdSc', txs_traintest= cv2._data)

In [123]:
prpt_withscaler.forecast('StdSc', forecast_dict= r.forecast_dict)

C:\Studying\Project_Nextop\nextop-engine\nextop_engine\_alg\prophet.py:174: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  result_forecast['ds'], box=True, format= '%Y/%m/%d', exact=True


In [124]:
for i, case_dict in r.forecast_dict.items():
    if 'StdSc' in i:
        case_dict['forecast']['y'] = std_scaler.inverse_transform(case_dict['forecast']['y'])
        case_dict['forecast']['yhat'] = std_scaler.inverse_transform(case_dict['forecast']['yhat'])
        print(case_dict['forecast'])

          ds         y  temp_max  temp_min  rain_amount  seasonality_7,3  \
0 2017-11-27  117480.0       8.1       1.8          0.0                0   

            yhat  
0  111374.932138  
          ds         y  temp_max   temp_min  rain_amount  seasonality_7,3  \
0 2017-09-18  124903.0      27.1  19.200001          0.0                0   

            yhat  
0  114622.816156  
          ds         y   temp_max  temp_min  rain_amount  seasonality_7,3  \
0 2017-07-10  112780.0  25.299999      21.4        123.7                0   

            yhat  
0  115054.050945  
          ds        y  temp_max  temp_min  rain_amount  seasonality_7,3  \
0 2017-05-01  89340.0      26.6      15.7          0.0                0   

            yhat  
0  104812.552203  
          ds         y  temp_max  temp_min  rain_amount  seasonality_7,3  \
0 2017-02-20  107804.0       0.6      -6.4          0.0                0   

            yhat  
0  130491.447928  
          ds         y  temp_max  temp_min 

In [125]:
r.merge_result(m_name= ['_0', '_1', '_2', '_3', '_4'], m_title= 'StdSc')

{'forecast':           ds         y  temp_max  temp_min  rain_amount  seasonality_7,3  \
0 2017-11-24  120647.0       2.9      -2.5          0.1                4   
0 2017-11-25   67756.0       6.4       0.8         15.7                5   
0 2017-11-26     100.0       6.4      -2.4          0.0                6   
0 2017-11-27  117480.0       8.1       1.8          0.0                0   
0 2017-11-28  122192.0       9.8       4.2          1.2                1   
0 2017-11-29  120685.0       5.7      -3.2          0.0                2   
0 2017-11-30  108386.0       1.8      -7.1          0.0                3   

            yhat  
0  122258.427620  
0   61708.098274  
0      39.808392  
0  111374.932138  
0  119510.385377  
0  122016.991061  
0  121081.118055  }
{'forecast':           ds         y   temp_max   temp_min  rain_amount  seasonality_7,3  \
0 2017-09-15  146743.0  27.299999  17.200001          0.0                4   
0 2017-09-16   78827.0  26.100000  19.200001          0.

In [126]:
r.err_rate(m_name= 'StdSc')
r.print_err_rate()

RMSE with segment weekday_result: 13547.0090
MAPE with segment weekday_result: 36.0416
MAPE_with_std with segment weekday_result: 25.3174
MAPE_div_std with segment weekday_result: 140.7680
sMAPE with segment weekday_result: 11.8902


RMSE with segment StdSc__0_merged: 5935.1323
MAPE with segment StdSc__0_merged: 12.9516
MAPE_with_std with segment StdSc__0_merged: 20.7205
MAPE_div_std with segment StdSc__0_merged: 73.5047
sMAPE with segment StdSc__0_merged: 4.6430


RMSE with segment StdSc__1_merged: 12684.2129
MAPE with segment StdSc__1_merged: 21.1217
MAPE_with_std with segment StdSc__1_merged: 26.3477
MAPE_div_std with segment StdSc__1_merged: 216.6725
sMAPE with segment StdSc__1_merged: 11.1406


RMSE with segment StdSc__2_merged: 7243.0942
MAPE with segment StdSc__2_merged: 48.8470
MAPE_with_std with segment StdSc__2_merged: 21.2731
MAPE_div_std with segment StdSc__2_merged: 82.2544
sMAPE with segment StdSc__2_merged: 5.0665


RMSE with segment StdSc__3_merged: 25560.9184
MAPE with

In [127]:
r.avg_result(m_name= 'StdSc')

{'forecast': [          ds         y  temp_max  temp_min  rain_amount  seasonality_7,3  \
0 2017-11-24  120647.0       2.9      -2.5          0.1                4   
0 2017-11-25   67756.0       6.4       0.8         15.7                5   
0 2017-11-26     100.0       6.4      -2.4          0.0                6   
0 2017-11-27  117480.0       8.1       1.8          0.0                0   
0 2017-11-28  122192.0       9.8       4.2          1.2                1   
0 2017-11-29  120685.0       5.7      -3.2          0.0                2   
0 2017-11-30  108386.0       1.8      -7.1          0.0                3   

            yhat  
0  122258.427620  
0   61708.098274  
0      39.808392  
0  111374.932138  
0  119510.385377  
0  122016.991061  
0  121081.118055  ,           ds         y   temp_max   temp_min  rain_amount  seasonality_7,3  \
0 2017-09-15  146743.0  27.299999  17.200001          0.0                4   
0 2017-09-16   78827.0  26.100000  19.200001          0.0           

In [128]:
r.print_err_rate()

RMSE with segment weekday_result: 13547.0090
MAPE with segment weekday_result: 36.0416
MAPE_with_std with segment weekday_result: 25.3174
MAPE_div_std with segment weekday_result: 140.7680
sMAPE with segment weekday_result: 11.8902


RMSE with segment StdSc_result: 14082.6801
MAPE with segment StdSc_result: 47.9085
MAPE_with_std with segment StdSc_result: 25.6671
MAPE_div_std with segment StdSc_result: 136.3199
sMAPE with segment StdSc_result: 12.4593




In [129]:
df_raw= ft_c.xlsx_opener(varr.PJ_DIR+varr.DF_DIR, INPUT_FILENAME)
df_temp= ft_c.xlsx_opener(varr.PJ_DIR+varr.TEMP_DATA_DIR, 'temp_data_merged.xlsx')

In [130]:
y_col= df_raw['raw'].columns.values.tolist()

In [131]:
try:
    y_col.remove('ds')
except ValueError:
    pass

In [132]:
x_col= df_temp['raw'].columns.values.tolist()

In [133]:
df_raw['raw']['ds']= pd.to_datetime(df_raw['raw']['ds'], box=True, format= '%Y/%m/%d', exact=True)
df_temp['raw']['ds']= pd.to_datetime(df_temp['raw']['ds'], box=True, format= '%Y/%m/%d', exact=True)
df_raw['raw']= pd.merge(df_raw['raw'], df_temp['raw'], how='inner', on='ds')

In [134]:
rbt_scaler= RobustScaler()

In [135]:
df_raw['raw']['y']= rbt_scaler.fit_transform(df_raw['raw']['y'].reshape(-1,1))

C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: reshape is deprecated and will raise in a subsequent release. Please use .values.reshape(...) instead
  """Entry point for launching an IPython kernel.


In [136]:
df_raw['raw']

,ds,y,temp_max,temp_min,rain_amount
0,2010-07-01,-0.160377,30.500000,24.900000,0.0
1,2010-07-02,-0.163287,25.799999,22.700001,70.0
2,2010-07-03,-0.836311,25.600000,22.600000,1.0
3,2010-07-04,-1.866338,29.700001,23.100000,0.0
4,2010-07-05,-0.476201,30.600000,21.799999,0.0
5,2010-07-06,-0.059273,31.000000,22.400000,0.0
6,2010-07-07,-0.140764,29.600000,21.900000,0.0
7,2010-07-08,-0.283310,29.000000,21.299999,0.0
8,2010-07-09,-0.147867,29.299999,21.799999,0.0
9,2010-07-10,-0.813006,29.100000,22.900000,0.0


In [137]:
ft_c.add_division(df_raw, 'raw', 7, start_num= 3)

In [138]:
ft_c.divide_by(df_raw, 'raw', 'seasonality_7,3')

In [139]:
df_raw

{(0,):              ds         y   temp_max   temp_min  rain_amount  seasonality_7,3
 4    2010-07-05 -0.476201  30.600000  21.799999          0.0                0
 11   2010-07-12 -0.419057  28.299999  22.200001          0.0                0
 18   2010-07-19 -0.285786  29.299999  24.200001          0.0                0
 25   2010-07-26 -0.244193  27.600000  23.400000          6.0                0
 32   2010-08-02 -0.427527  31.600000  24.000000          1.0                0
 39   2010-08-09 -0.418818  32.799999  26.400000          0.0                0
 46   2010-08-16 -0.348099  26.500000  21.000000          0.0                0
 53   2010-08-23 -0.337782  28.900000  23.500000         13.0                0
 60   2010-08-30 -0.410478  29.799999  22.299999          0.0                0
 67   2010-09-06 -0.272558  29.700001  23.000000          0.0                0
 74   2010-09-13 -0.245670  27.400000  21.100000          0.0                0
 81   2010-09-20 -0.668766  25.400000  19.4000

In [140]:
cv3= Cross_Validation(df_raw)

In [141]:
cv3.slice('y', x_col, forecastday= varr.FORECASTDAY//7)

In [142]:
cv3._data

{'(0,)_0': {'test':              ds         y  temp_max  temp_min  rain_amount  seasonality_7,3
  2706 2017-11-27  0.662576       8.1       1.8          0.0                0,
  'testX':              ds  temp_max  temp_min  rain_amount
  2706 2017-11-27       8.1       1.8          0.0,
  'testY': 2706    0.662576
  Name: y, dtype: float64,
  'train':              ds         y   temp_max   temp_min  rain_amount  seasonality_7,3
  4    2010-07-05 -0.476201  30.600000  21.799999          0.0                0
  11   2010-07-12 -0.419057  28.299999  22.200001          0.0                0
  18   2010-07-19 -0.285786  29.299999  24.200001          0.0                0
  25   2010-07-26 -0.244193  27.600000  23.400000          6.0                0
  32   2010-08-02 -0.427527  31.600000  24.000000          1.0                0
  39   2010-08-09 -0.418818  32.799999  26.400000          0.0                0
  46   2010-08-16 -0.348099  26.500000  21.000000          0.0                0
  53   20

In [143]:
prpt_withrobust= Prophet_timeseries(cv= True)

In [144]:
prpt_withrobust.add_model('RbtSc', 'week', regressor= ['temp_max', 'temp_min', 'rain_amount'])

In [145]:
prpt_withrobust.fit('RbtSc', txs_traintest= cv3._data)

In [146]:
prpt_withrobust.forecast('RbtSc', forecast_dict= r.forecast_dict)

C:\Studying\Project_Nextop\nextop-engine\nextop_engine\_alg\prophet.py:174: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  result_forecast['ds'], box=True, format= '%Y/%m/%d', exact=True


In [147]:
for i, case_dict in r.forecast_dict.items():
    if 'RbtSc' in i:
        case_dict['forecast']['y'] = rbt_scaler.inverse_transform(case_dict['forecast']['y'].reshape(-1,1))
        case_dict['forecast']['yhat'] = rbt_scaler.inverse_transform(case_dict['forecast']['yhat'].reshape(-1,1))
        print(case_dict['forecast'])

C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: FutureWarning: reshape is deprecated and will raise in a subsequent release. Please use .values.reshape(...) instead
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: FutureWarning: reshape is deprecated and will raise in a subsequent release. Please use .values.reshape(...) instead
  after removing the cwd from sys.path.


          ds         y  temp_max  temp_min  rain_amount  seasonality_7,3  \
0 2017-11-27  117480.0       8.1       1.8          0.0                0   

            yhat  
0  112048.529739  
          ds         y  temp_max   temp_min  rain_amount  seasonality_7,3  \
0 2017-09-18  124903.0      27.1  19.200001          0.0                0   

            yhat  
0  114520.974348  
          ds         y   temp_max  temp_min  rain_amount  seasonality_7,3  \
0 2017-07-10  112780.0  25.299999      21.4        123.7                0   

            yhat  
0  115761.442942  
          ds        y  temp_max  temp_min  rain_amount  seasonality_7,3  \
0 2017-05-01  89340.0      26.6      15.7          0.0                0   

            yhat  
0  105653.338894  
          ds         y  temp_max  temp_min  rain_amount  seasonality_7,3  \
0 2017-02-20  107804.0       0.6      -6.4          0.0                0   

            yhat  
0  132823.776511  
          ds         y  temp_max  temp_min 

In [148]:
r.merge_result(m_name= ['_0', '_1', '_2', '_3', '_4'], m_title= 'RbtSc')

{'forecast':           ds         y  temp_max  temp_min  rain_amount  seasonality_7,3  \
0 2017-11-24  120647.0       2.9      -2.5          0.1                4   
0 2017-11-25   67756.0       6.4       0.8         15.7                5   
0 2017-11-26     100.0       6.4      -2.4          0.0                6   
0 2017-11-27  117480.0       8.1       1.8          0.0                0   
0 2017-11-28  122192.0       9.8       4.2          1.2                1   
0 2017-11-29  120685.0       5.7      -3.2          0.0                2   
0 2017-11-30  108386.0       1.8      -7.1          0.0                3   

            yhat  
0  122862.349595  
0   61330.390527  
0      31.682509  
0  112048.529739  
0  120227.159419  
0  122250.611343  
0  121779.007780  }
{'forecast':           ds         y   temp_max   temp_min  rain_amount  seasonality_7,3  \
0 2017-09-15  146743.0  27.299999  17.200001          0.0                4   
0 2017-09-16   78827.0  26.100000  19.200001          0.

In [149]:
r.err_rate(m_name= 'RbtSc')
r.print_err_rate()

RMSE with segment weekday_result: 13547.0090
MAPE with segment weekday_result: 36.0416
MAPE_with_std with segment weekday_result: 25.3174
MAPE_div_std with segment weekday_result: 140.7680
sMAPE with segment weekday_result: 11.8902


RMSE with segment StdSc_result: 14082.6801
MAPE with segment StdSc_result: 47.9085
MAPE_with_std with segment StdSc_result: 25.6671
MAPE_div_std with segment StdSc_result: 136.3199
sMAPE with segment StdSc_result: 12.4593


RMSE with segment RbtSc__0_merged: 6110.7185
MAPE with segment RbtSc__0_merged: 14.2175
MAPE_with_std with segment RbtSc__0_merged: 20.7219
MAPE_div_std with segment RbtSc__0_merged: 72.8372
sMAPE with segment RbtSc__0_merged: 4.7146


RMSE with segment RbtSc__1_merged: 11866.5795
MAPE with segment RbtSc__1_merged: 20.9501
MAPE_with_std with segment RbtSc__1_merged: 25.5903
MAPE_div_std with segment RbtSc__1_merged: 227.5877
sMAPE with segment RbtSc__1_merged: 10.4761


RMSE with segment RbtSc__2_merged: 7193.6637
MAPE with segment RbtS

In [150]:
r.avg_result(m_name= 'RbtSc')

{'forecast': [          ds         y  temp_max  temp_min  rain_amount  seasonality_7,3  \
0 2017-11-24  120647.0       2.9      -2.5          0.1                4   
0 2017-11-25   67756.0       6.4       0.8         15.7                5   
0 2017-11-26     100.0       6.4      -2.4          0.0                6   
0 2017-11-27  117480.0       8.1       1.8          0.0                0   
0 2017-11-28  122192.0       9.8       4.2          1.2                1   
0 2017-11-29  120685.0       5.7      -3.2          0.0                2   
0 2017-11-30  108386.0       1.8      -7.1          0.0                3   

            yhat  
0  122862.349595  
0   61330.390527  
0      31.682509  
0  112048.529739  
0  120227.159419  
0  122250.611343  
0  121779.007780  ,           ds         y   temp_max   temp_min  rain_amount  seasonality_7,3  \
0 2017-09-15  146743.0  27.299999  17.200001          0.0                4   
0 2017-09-16   78827.0  26.100000  19.200001          0.0           

In [151]:
r.print_err_rate()

RMSE with segment weekday_result: 13547.0090
MAPE with segment weekday_result: 36.0416
MAPE_with_std with segment weekday_result: 25.3174
MAPE_div_std with segment weekday_result: 140.7680
sMAPE with segment weekday_result: 11.8902


RMSE with segment StdSc_result: 14082.6801
MAPE with segment StdSc_result: 47.9085
MAPE_with_std with segment StdSc_result: 25.6671
MAPE_div_std with segment StdSc_result: 136.3199
sMAPE with segment StdSc_result: 12.4593


RMSE with segment RbtSc_result: 14178.3739
MAPE with segment RbtSc_result: 47.8878
MAPE_with_std with segment RbtSc_result: 25.6062
MAPE_div_std with segment RbtSc_result: 139.3700
sMAPE with segment RbtSc_result: 12.5734


